In [2]:
import pandas as pd
import pyarrow.parquet as pq

# Uploading the two df

In [4]:

table = pq.read_table("../sourcedata/clips_metadata_from_im.parquet")
df_im = table.to_pandas(types_mapper=None)
df_base = pd.read_csv("../sourcedata/clips_metadata_with_patterns.csv")

df_im = df_im.rename(columns={"StateClipCode": "ClipCode"})

print(df_base.columns)
print('Base: ', df_base.shape)
print('Json imitation: ', df_im.shape)

/tmp/ipykernel_44668/2449628635.py:3: DtypeWarning: Columns (1,6,8,26,31,32,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_base = pd.read_csv("../sourcedata/clips_metadata_with_patterns.csv")


Index(['Model', 'StateFileName', 'LevelFullName', 'Scene', 'ClipCode',
       'TotalFrames', 'Bk2Filepath', 'GameName', 'Subject', 'World', 'Level',
       'Duration', 'Cleared', 'ScoreGained', 'X_Traveled', 'Lives_lost',
       'Hits_taken', 'Enemies_killed', 'Powerups_collected',
       'Bricks_destroyed', 'CoinsGained', 'Average_speed', 'Session', 'Run',
       'StartFrame', 'EndFrame', 'Bk2File', 'SceneFullName', 'IndexInRun',
       'IndexGlobal', 'IndexLevel', 'Phase', 'Phase_stage', 'Pattern_Enemy',
       'Pattern_2-Horde', 'Pattern_3-Horde', 'Pattern_4-Horde', 'Pattern_Roof',
       'Pattern_Gap', 'Pattern_Multiple gaps', 'Pattern_Variable gaps',
       'Pattern_Gap enemy', 'Pattern_Pillar gap', 'Pattern_Pipe valley',
       'Pattern_Empty valley', 'Pattern_Enemy valley', 'Pattern_Roof valley',
       'Pattern_2-Path', 'Pattern_3-Path', 'Pattern_Risk/Reward',
       'Pattern_Stair up', 'Pattern_Stair down', 'Pattern_Empty stair valley',
       'Pattern_Enemy stair valley', 'Pa

In [5]:
print(df_base['ClipCode'].size)
print(df_im['ClipCode'].size)
print(df_base['ClipCode'][df_base['Model']=='human'].size)


336846
280705
56141


In [6]:
nb_clipcodes_base = df_base['ClipCode'].size
nb_clipcodes_json = df_im['ClipCode'].size
nb_tries_human = df_base['ClipCode'][df_base['Model']=='human'].size

print(nb_clipcodes_base/nb_tries_human)
print(nb_clipcodes_json/nb_tries_human)

6.0
5.0


nb_clipcodes_json/nb_tries_human should be equal to 5, because each imitation agent only play the .state of the subject it is imitating, and we have 5 checkpoints per agents (nb_tries_human*5).

In [7]:
df_base["ClipCode_str"] = df_base["ClipCode"].astype(str).str.zfill(14)
df_im["ClipCode_str"] = df_im["ClipCode"].astype(str).str.zfill(14)

missing_clipcode = set(df_base["ClipCode_str"]) - set(df_im["ClipCode_str"])
print(missing_clipcode)

df_missing = df_im['ClipCode'].value_counts()[df_im['ClipCode'].value_counts() < 5]
df_missing.head()

set()


Series([], Name: count, dtype: int64)

We are not missing 20 .json (clips) from sub-03 ses-014, 5\*{'01403050001610', '01403050003350', '01403050002418'}, 1\*01403050004489, and 4\*01403050004141. Resulting in a defficit of 20 clips. Exactly as show bellow

In [8]:
nb_tries_human*5 - nb_clipcodes_json

0

# Figures

In [9]:
df_im['Scene'] = df_im['Scene'].astype(int)
df = pd.concat([df_base, df_im], ignore_index=True)
level = 'w1l2'
scene = 1
subject = 'sub-01'
scene_fullname = f"{level}s{scene}"

print(df['Scene'].dtype)

int64


In [112]:
df

,Model,StateFileName,LevelFullName,Scene,ClipCode,TotalFrames,Bk2Filepath,GameName,Subject,World,...,Pattern_Gap stair valley,Pattern_Reward,Pattern_Moving platform,Pattern_Flagpole,Pattern_Beginning,Pattern_Bonus zone,Pattern_Waterworld,Learning_Phase,ClipCode_str,json_path
0,human,NaN,w1l3,0,103100000111,2628,NaN,SuperMarioBros-Nes,sub-01,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Late discovery,00103100000111,NaN
1,human,NaN,w1l1,10,101000003476,4831,NaN,SuperMarioBros-Nes,sub-01,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Early discovery,00101000003476,NaN
2,human,NaN,w1l3,6,103010001275,2528,NaN,SuperMarioBros-Nes,sub-01,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Early discovery,00103010001275,NaN
3,human,NaN,w2l1,13,104000002344,3354,NaN,SuperMarioBros-Nes,sub-01,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Early discovery,00104000002344,NaN
4,human,NaN,w1l2,6,102010002114,3090,NaN,SuperMarioBros-Nes,sub-01,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Early discovery,00102010002114,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617546,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w4l3,2,00803030001282,165,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,im_sub-06,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-06_epoch=0-step=4000,00803030001282,../../../github/mario_curiosity.scene_agents/o...
617547,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w1l3,8,00803020004180,154,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,im_sub-06,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-06_epoch=0-step=4000,00803020004180,../../../github/mario_curiosity.scene_agents/o...
617548,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w5l3,3,00802010001104,477,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,im_sub-06,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-06_epoch=0-step=4000,00802010001104,../../../github/mario_curiosity.scene_agents/o...
617549,sub-06_epoch=0-step=4000,scene_clips/sub-06/ses-008/beh/savestates/sub-...,w6l3,6,00801010002778,4743,sub-06_epoch=0-step=4000/sub-06/ses-008/beh/bk...,SuperMarioBros-Nes,im_sub-06,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-06_epoch=0-step=4000,00801010002778,../../../github/mario_curiosity.scene_agents/o...


In [11]:
mask_im = df['Model'].str.startswith('sub-0')
df.loc[mask_im, 'Learning_Phase'] = df.loc[mask_im, 'Model'].str[:]
df.loc[mask_im, 'Subject'] = "im_"+df.loc[mask_im, 'Model'].str[:6]

mask_ppo = df['Model'].str.startswith('ep')
df.loc[mask_ppo, 'Learning_Phase'] = df.loc[mask_ppo, 'Model'].str[:]
df.loc[mask_ppo, 'Subject'] = "ppo"

print(df['Learning_Phase'].unique())
print(df['Subject'].unique())

['Late discovery' 'Early discovery' 'Early practice' 'Late practice'
 'ep-20' 'ep-2000' 'ep-4000' 'ep-6000' 'ep-8000'
 'sub-06_epoch=0-step=2000' 'sub-03_epoch=1-step=14908'
 'sub-01_epoch=0-step=6500' 'sub-03_epoch=0-step=4000'
 'sub-01_epoch=0-step=2000' 'sub-02_epoch=0-step=8000'
 'sub-03_epoch=0-step=500' 'sub-05_epoch=0-step=4000'
 'sub-06_epoch=0-step=5500' 'sub-05_epoch=0-step=1500'
 'sub-05_epoch=0-step=3000' 'sub-03_epoch=1-step=11408'
 'sub-05_epoch=0-step=500' 'sub-01_epoch=0-step=500'
 'sub-02_epoch=0-step=3000' 'sub-06_epoch=0-step=500'
 'sub-03_epoch=0-step=7500' 'sub-02_epoch=0-step=5500'
 'sub-05_epoch=0-step=5000' 'sub-06_epoch=0-step=7000'
 'sub-02_epoch=0-step=500' 'sub-01_epoch=0-step=3500'
 'sub-02_epoch=0-step=10000' 'sub-01_epoch=0-step=5000'
 'sub-06_epoch=0-step=4000']
['sub-01' 'sub-02' 'sub-03' 'sub-05' 'sub-06' 'ppo' 'im_sub-06'
 'im_sub-03' 'im_sub-01' 'im_sub-02' 'im_sub-05']


In [ ]:
# %pip install plotly ipywidgets pillow

import os
import numpy as np
import pandas as pd
from PIL import Image

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import sample_colorscale

import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

# ---------- DATA ----------
# Keep your existing df; otherwise you can load:
# df = pd.read_csv("/mnt/data/tmp_filtered_clips.csv")

df_human = df[df["Model"] == "human"].copy()
df_ppos = df[df["Model"] != "human"].copy()
sub_order = np.concat([df_human["Subject"].sort_values().unique(),
                        df_ppos["Subject"].sort_values().unique()])
## top-right
phase_order = ["discovery", "practice"]
phase_colors = dict(zip(phase_order, sample_colorscale("Viridis", [0.33,0.66])))
phase_position_cl = dict(discovery = "bottom left", practice="bottom right")
dict_hum = {'sub-01': ['discovery', 'practice'],
            'sub-02': ['discovery', 'practice'],
            'sub-03': ['discovery', 'practice'],
            'sub-05': ['discovery', 'practice'],
            'sub-06': ['discovery', 'practice']}
# bottom-left
human_phases = ["Early discovery", "Late discovery", "Early practice", "Late practice"]
ppos_phases = ['sub-01_epoch=0-step=500', 'sub-01_epoch=0-step=2000', 'sub-01_epoch=0-step=3500', 'sub-01_epoch=0-step=5000', 'sub-01_epoch=0-step=6500',
                    'sub-02_epoch=0-step=500', 'sub-02_epoch=0-step=3000', 'sub-02_epoch=0-step=5500', 'sub-02_epoch=0-step=8000', 'sub-02_epoch=0-step=10000',
                    'sub-03_epoch=0-step=500', 'sub-03_epoch=0-step=4000', 'sub-03_epoch=0-step=7500', 'sub-03_epoch=1-step=11408', 'sub-03_epoch=1-step=14908',
                    'sub-05_epoch=0-step=500', 'sub-05_epoch=0-step=1500', 'sub-05_epoch=0-step=3000', 'sub-05_epoch=0-step=4000', 'sub-05_epoch=0-step=5000',
                    'sub-06_epoch=0-step=500', 'sub-06_epoch=0-step=2000', 'sub-06_epoch=0-step=4000', 'sub-06_epoch=0-step=5500', 'sub-06_epoch=0-step=7000',
                    'ep-20', 'ep-2000', 'ep-4000', 'ep-6000', 'ep-8000']

dict_all = {'sub-01': ['Early discovery', 'Late discovery', 'Early practice', 'Late practice'],
            'sub-02': ['Early discovery', 'Late discovery', 'Early practice', 'Late practice'],
            'sub-03': ['Early discovery', 'Late discovery', 'Early practice', 'Late practice'],
            'sub-05': ['Early discovery', 'Late discovery', 'Early practice', 'Late practice'],
            'sub-06': ['Early discovery', 'Late discovery', 'Early practice', 'Late practice'],
            'im_sub-01': ['sub-01_epoch=0-step=500', 'sub-01_epoch=0-step=2000', 'sub-01_epoch=0-step=3500', 'sub-01_epoch=0-step=5000', 'sub-01_epoch=0-step=6500'],
            'im_sub-02': ['sub-02_epoch=0-step=500', 'sub-02_epoch=0-step=3000', 'sub-02_epoch=0-step=5500', 'sub-02_epoch=0-step=8000', 'sub-02_epoch=0-step=10000'],
            'im_sub-03': ['sub-03_epoch=0-step=500', 'sub-03_epoch=0-step=4000', 'sub-03_epoch=0-step=7500', 'sub-03_epoch=1-step=11408', 'sub-03_epoch=1-step=14908'],
            'im_sub-05': ['sub-05_epoch=0-step=500', 'sub-05_epoch=0-step=1500', 'sub-05_epoch=0-step=3000', 'sub-05_epoch=0-step=4000', 'sub-05_epoch=0-step=5000'],
            'im_sub-06': ['sub-06_epoch=0-step=500', 'sub-06_epoch=0-step=2000', 'sub-06_epoch=0-step=4000', 'sub-06_epoch=0-step=5500', 'sub-06_epoch=0-step=7000'],
            'ppo': ['ep-20', 'ep-2000', 'ep-4000', 'ep-6000', 'ep-8000']}

phase_order_bottom_fig = human_phases + ppos_phases

phase_color_human= sample_colorscale("Viridis", [0.00,0.33, 0.66, 1.00])
check_point_color = sample_colorscale("magma", [0.2,0.4,0.6,0.8,1.0])
bottom_phase_colors = dict(zip(phase_order_bottom_fig, phase_color_human+check_point_color*6))


'''# Z-score across the entire human dataset (per metric)
means = df_human[metrics].mean(numeric_only=True)
stds  = df_human[metrics].std(numeric_only=True).replace(0, np.nan)
z_cols = []
for m in metrics:
    zc = f"{m}_z"
    z_cols.append(zc)
    df_human[zc] = (df_human[m] - means[m]) / stds[m]
'''

# ---------- HELPERS ----------
def scenes_for_level(level):
    return sorted(df_human.loc[df_human["LevelFullName"] == level, "Scene"].dropna().unique().tolist())

def subjects_for(level, scene):
    m = (df_human["LevelFullName"] == level) & (df_human["Scene"] == scene)
    return sorted(df_human.loc[m, "Subject"].dropna().unique().tolist())

def get_subset(level, scene):
       m = (
        (df["LevelFullName"] == level) &
        (df["Scene"] == scene))
       return df.loc[m].copy()

def load_scene_image(level, scene):
    scene_fullname = f"{level}s{scene}"
    root = Path.cwd().parent
    path = os.path.join(root, 'sourcedata', 'mario_backgrounds', 'scene_backgrounds', f"{scene_fullname}.png")
    if os.path.exists(path):
        im = Image.open(path).convert("RGBA")
        return np.array(im), path
    return None, path

def compute_delta_tot(df, phase_to_ckpt, metric="Cleared"):
    # moyenne clearance par checkpoint
    df_tmp = df.copy()
    #print(df_tmp)
    df_tmp["chk"] = df_tmp["Learning_Phase"].map(phase_to_ckpt)
    #print(phase_to_ckpt)
    df_chk =df_tmp.groupby(["Subject", "chk"])[metric].mean().reset_index(name="clearance")
    #print(df_chk)

    idx_max = df_chk.groupby("Subject")["chk"].idxmax()
    df_max = df_chk.loc[idx_max, ["Subject", "clearance"]].rename(columns={"clearance": "clearance_max"})    
    idx_min = df_chk.groupby("Subject")["chk"].idxmin()
    df_min = df_chk.loc[idx_min, ["Subject", "clearance"]].rename(columns={"clearance": "clearance_min"})

    res = df_min.merge(df_max, on="Subject")
    res['delta_tot'] = res["clearance_max"] - res["clearance_min"]
    return res


def compute_delta_long(df, phase_to_ckpt):
    # copie + map phases -> chk
    df_tmp = df.copy()
    df_tmp["chk"] = df_tmp["Learning_Phase"].map(phase_to_ckpt)

    # mean clearance per Subject, chk
    df_chk = (
        df_tmp.groupby(["Subject", "chk"])["Cleared"]
        .mean()
        .reset_index(name="clearance")
    )

    # all checkpoints sorted (assure que ce sont comparables)
    all_chks = sorted(df_chk["chk"].unique())

    # pivot : Subject x chk (wide)
    df_wide = df_chk.pivot(index="Subject", columns="chk", values="clearance")

    # reindex columns (checkpoints) pour garantir présence de toutes les colonnes
    df_wide = df_wide.reindex(columns=all_chks)

    # reindex rows pour garantir tous les sujets (optionnel : si tu veux tous les sujets connus ailleurs)
    # df_wide = df_wide.reindex(sorted(df_wide.index))

    # shift vers la gauche : clearance_{k+1} aligned with column k
    df_shifted = df_wide.shift(-1, axis=1)

    # delta = shifted - original  -> delta at chk k = chk{k+1} - chk{k}
    df_delta = df_shifted - df_wide

    # by construction, last column becomes NaN (no chk{k+1} for last)
    # ensure columns remain in same order
    df_delta = df_delta[all_chks]

    # convert to long format and keep clearance too
    df_long_clear = df_wide.reset_index().melt(id_vars="Subject", var_name="chk", value_name="clearance")
    df_long_delta = df_delta.reset_index().melt(id_vars="Subject", var_name="chk", value_name="delta")

    # merge clearance + delta
    df_long = pd.merge(df_long_clear, df_long_delta, on=["Subject", "chk"])

    # optional : sort
    df_long = df_long.sort_values(["Subject", "chk"]).reset_index(drop=True)

    return df_long


def make_dashboard(level, scene):
    df_wls = get_subset(level, scene)

    df_human = df_wls[df_wls["Model"] == "human"].copy()
    df_ppos = df_wls[df_wls["Model"] != "human"].copy()

    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type": "xy"}, {"type": "xy"}], [{"type": "xy"}, {"type": "xy"}]],
        column_widths=[0.5, 0.5],
        horizontal_spacing=0.06,
        subplot_titles=("Scene Background", " Num. Clips per Learning Phase", "Clearance Through Learning-Phases", "Speed Through Learning-Phases")
    )

    ########################################
    # Top-Left: non-interactive background#
    ########################################

    img_arr, img_path = load_scene_image(level, scene)
    if img_arr is not None:
        fig.add_trace(go.Image(z=img_arr, hoverinfo="skip"), row=1, col=1)
    else:
        fig.add_annotation(row=1, col=1, x=0.5, y=0.5, xref="x1", yref="y1",
                           text=f"No image found:<br>{img_path}", showarrow=False)
    fig.update_xaxes(visible=False, fixedrange=True, row=1, col=1)
    fig.update_yaxes(visible=False, fixedrange=True, row=1, col=1)

    ###############################################
    # Top-Right: bar (counts) with outside labels#
    ###############################################    

    df_count_tries = (
        df_human.groupby(["Subject", "Phase"])["ClipCode"]
        .count()
        .reset_index(name="count")
    )
    df_counts_cleared = (
        df_human.groupby(["Subject", "Phase"])["Cleared"]
        .mean()
        .reset_index(name="clearance")
    )
    show_legend = False
    legende_is_first_time = True
    for sub in dict_hum.keys():
        sub_phases = df_count_tries[df_count_tries["Subject"] == sub]["Phase"].unique()
        for phase in dict_hum[sub]:
            df_phase = df_count_tries[(df_count_tries["Phase"] == phase) & (df_count_tries["Subject"] == sub)]
            df_phase = df_phase.merge(
                df_counts_cleared[df_counts_cleared["Phase"] == phase],
                on=["Subject", "Phase"], how="left"
            )

            if df_phase.empty:
                df_phase = pd.DataFrame({
                    "Subject": [sub],
                    "Phase": [phase],
                    "count": [0],
                    "clearance": [0.0]
                })

            if set(phase_order).issubset(set(sub_phases)):
                if legende_is_first_time:
                    show_legend = True
                    legende_is_first_time = False

            fig.add_trace(
                go.Bar(
                    x=df_phase["Subject"],
                    y=df_phase["count"],
                    name=phase,
                    marker_color=phase_colors[phase],
                    textfont=dict(
                        size=13,
                        color='rgb(68, 1, 84)',       
                        family="Arial Black"
                    ),
                    text=df_phase["count"],
                    textposition="outside",
                    offsetgroup=phase,
                    cliponaxis=False,
                    hovertemplate="Phase: %{name}<br>Value: %{y}<extra></extra>",
                    showlegend=show_legend
                    ),
                row=1, col=2
            )

            fig.add_trace(
            go.Scatter(
                x=df_phase["Subject"],
                y=df_phase["clearance"] * df_phase["count"],
                mode="markers+text",
                offsetgroup=phase,
                showlegend=show_legend,
                text=(df_phase["clearance"] * 100).round(0).astype(str) + "%",
                textposition=phase_position_cl[phase],
                textfont=dict(
                    size=13,
                    color='rgb(68, 1, 84)',       
                    family="Arial Black"
                    ),
                marker=dict(
                    size=10,
                    color=phase_colors[phase],
                    symbol="diamond-wide",
                    line=dict(width=1, color='rgb(68, 1, 84)')
                    ),
        #marker_symbol='line-ew',
                name=f"Success Rate at {phase} Phase ",
                ),
                row=1, col=2
            )
        show_legend = False

    fig.update_layout(barmode="group")
    fig.update_yaxes(title_text="N tries", row=1, col=2, range=[-3, df_count_tries["count"].max() * 1.2])
    fig.update_xaxes(title_text="Learning Phase", row=2, col=1)

    ###############################
    # Bottom Left: bar (clerance) #
    ###############################
    df_clearance = df_wls.groupby(["Subject", "Learning_Phase"])["Cleared"].mean().reset_index(name="clearance")

    
    phase_to_human = {p: i % 4 for i, p in enumerate(human_phases)}
    phase_to_checkpoint = {p: i % 5 for i, p in enumerate(ppos_phases)}
    phase_to_all = {**phase_to_human, **phase_to_checkpoint}
    
    
    delta_human = compute_delta_long(df_human, phase_to_human)
    #hum_order = df_human['Learning_Phase'].sort_values().unique()
    #map_hum = {idx: subj for idx, subj in enumerate(hum_order)}
    #delta_human['Learning_Phase'] = delta_human.index.map(map_hum)

    delta_agent = compute_delta_long(df_ppos, phase_to_checkpoint)
    #ppos_order = df_ppos['Learning_Phase'].sort_values().unique()
    #map_agent = {idx: subj for idx, subj in enumerate(ppos_order)}
    #delta_agent['Learning_Phase'] = delta_agent.index.map(map_agent)

    #df_delta = pd.concat([delta_human, delta_agent], ignore_index=True)


    for sub in dict_all.keys():
        for idx, i in enumerate(dict_all[sub]):
            df_phase = df_clearance[(df_clearance["Learning_Phase"] == i) & (df_clearance["Subject"] == sub)]
            if df_phase.empty:
                df_phase = pd.DataFrame({
                    "Subject": [sub],
                    "Learning_Phase": [i],
                    "clearance": [0.0]
                })
            chk = phase_to_all[i]   # <-- NEW: group phases into 5 groups
            fig.add_trace(
                go.Bar(
                    x=df_phase["Subject"],
                    y=df_phase["clearance"],
                    name=i,
                    marker_color=bottom_phase_colors[i],
                    offsetgroup=f"chk{chk}",
                    text=(df_phase["clearance"] * 100).round(0).astype(str) + "%",
                    textposition="inside",
                    cliponaxis=False,
                    hovertemplate="Learning_Phase: %{name}<br>Value: %{y:.2f}<extra></extra>",
                    showlegend=False
                ),
                row=2, col=1
            )
    
    fig.update_yaxes(title_text="Clearance Rate", row=2, col=1, range=[-0.1, 1.2])

    simple_names = human_phases+["Checkpoint 1", "Checkpoint 2", "Checkpoint 3", "Checkpoint 4", "Checkpoint 5"]

    # Ajouter une entrée de légende par couleur (invisible dans le graphe)
    for label, color in zip(simple_names, phase_color_human+check_point_color):
        fig.add_trace(
            go.Scatter(
                x=[None], y=[None],      # <-- ne s'affiche pas dans la figure
                mode="markers",
                marker=dict(size=12, color=color),
                name=label,              # <-- ce qui apparaît dans la légende
                showlegend=True,
            ),
            row=2, col=1
        )
    
    delta_tot_agent = compute_delta_tot(df_ppos, phase_to_checkpoint)
    delta_tot_hum = compute_delta_tot(df_human, phase_to_human)
    df_delta_tot = pd.concat([delta_tot_hum, delta_tot_agent], ignore_index=True)

    for sub in df_delta_tot['Subject'].sort_values().unique():
        df_sub = df_delta_tot[df_delta_tot['Subject'] == sub]
        fig.add_trace(
            go.Scatter(
                x=df_sub['Subject'],
                y=[1],
                mode='text',
                text=(df_sub['delta_tot']).round(2).astype(str) + " d",
                textposition="top center",
                textfont=dict(
                size=13,
                color='rgb(68, 1, 84)',       
                family="Arial Black"
                ),
                name=f"Delta Total Clearance for {sub}",
                showlegend=False
            ),
            row=2, col=1
        )

    ###############################
    # Bottom Right: bar (speed) #
    ###############################

    df_clearance = df_wls.groupby(["Subject", "Learning_Phase"])["Average_speed"].mean().reset_index(name="Average_speed")
    
    phase_to_human = {p: i % 4 for i, p in enumerate(human_phases)}
    phase_to_checkpoint = {p: i % 5 for i, p in enumerate(ppos_phases)}
    phase_to_all = {**phase_to_human, **phase_to_checkpoint}
    
    
    delta_human = compute_delta_long(df_human, phase_to_human)
    #hum_order = df_human['Learning_Phase'].sort_values().unique()
    #map_hum = {idx: subj for idx, subj in enumerate(hum_order)}
    #delta_human['Learning_Phase'] = delta_human.index.map(map_hum)

    delta_agent = compute_delta_long(df_ppos, phase_to_checkpoint)
    #ppos_order = df_ppos['Learning_Phase'].sort_values().unique()
    #map_agent = {idx: subj for idx, subj in enumerate(ppos_order)}
    #delta_agent['Learning_Phase'] = delta_agent.index.map(map_agent)

    #df_delta = pd.concat([delta_human, delta_agent], ignore_index=True)


    for sub in dict_all.keys():
        for idx, i in enumerate(dict_all[sub]):
            df_phase = df_clearance[(df_clearance["Learning_Phase"] == i) & (df_clearance["Subject"] == sub)]
            if df_phase.empty:
                df_phase = pd.DataFrame({
                    "Subject": [sub],
                    "Learning_Phase": [i],
                    "Average_speed": [0.0]
                })
            chk = phase_to_all[i]   # <-- NEW: group phases into 5 groups
            fig.add_trace(
                go.Bar(
                    x=df_phase["Subject"],
                    y=df_phase["Average_speed"],
                    name=i,
                    marker_color=bottom_phase_colors[i],
                    offsetgroup=f"chk{chk}",
                    text=(df_phase["Average_speed"] * 100).round(0).astype(str) + "%",
                    textposition="inside",
                    cliponaxis=False,
                    hovertemplate="Learning_Phase: %{name}<br>Value: %{y:.2f}<extra></extra>",
                    showlegend=False
                ),
                row=2, col=2
            )
    
    fig.update_yaxes(title_text="Average Speed", row=2, col=1, range=[-0.1, 1.2])

    simple_names = human_phases+["Checkpoint 1", "Checkpoint 2", "Checkpoint 3", "Checkpoint 4", "Checkpoint 5"]

    # Ajouter une entrée de légende par couleur (invisible dans le graphe)
    for label, color in zip(simple_names, phase_color_human+check_point_color):
        fig.add_trace(
            go.Scatter(
                x=[None], y=[None],      # <-- ne s'affiche pas dans la figure
                mode="markers",
                marker=dict(size=12, color=color),
                name=label,              # <-- ce qui apparaît dans la légende
                showlegend=False,
            ),
            row=2, col=2
        )
    
    delta_tot_agent = compute_delta_tot(df_ppos, phase_to_checkpoint, metric="Average_speed")
    delta_tot_hum = compute_delta_tot(df_human, phase_to_human, metric="Average_speed")
    df_delta_tot = pd.concat([delta_tot_hum, delta_tot_agent], ignore_index=True)

    for sub in df_delta_tot['Subject'].sort_values().unique():
        df_sub = df_delta_tot[df_delta_tot['Subject'] == sub]
        fig.add_trace(
            go.Scatter(
                x=df_sub['Subject'],
                y=[120],
                mode='text',
                text=(df_sub['delta_tot']).round(2).astype(str) + " d",
                textposition="top center",
                textfont=dict(
                size=13,
                color='rgb(68, 1, 84)',       
                family="Arial Black"
                ),
                name=f"Delta Total Clearance for {sub}",
                showlegend=False
            ),
            row=2, col=2
        )

    # Global layout tuned for clean initial render
    scene_fullname = f"{level}s{scene}"
    fig.update_layout(
        height=540,
        autosize=True,
        margin=dict(l=20, r=20, t=70, b=70),
        title_text=f"Level: {level} | Scene: {scene} ({scene_fullname})",
        legend=dict(orientation="h", y=-0.22, x=0.0),
        uniformtext_minsize=10, uniformtext_mode='hide',
        bargap=0.3,
    )
    # Shrink subplot titles, nudge slightly
    for ann in fig.layout.annotations:
        ann.font.size = 12
        ann.yshift = 6

    return fig

# ---------- WIDGETS ----------
levels = sorted(df_human["LevelFullName"].dropna().unique())

level_dd = widgets.Dropdown(
    options=levels, value=levels[0],
    description="Level:", layout=widgets.Layout(width="240px")
)

# Scene as SelectionSlider with exactly N positions (available scenes for the level)
def build_scene_slider(level):
    opts = scenes_for_level(level)
    if not opts:
        return widgets.SelectionSlider(options=[], value=None, description="Scene:", layout=widgets.Layout(width="500px"))
    # label each tick with the scene number; N positions = len(opts)
    return widgets.SelectionSlider(
        options=[(str(s), s) for s in opts],
        value=opts[0],
        description="Scene:",
        layout=widgets.Layout(width="500px"),
        continuous_update=False
    )

scene_slider = build_scene_slider(level_dd.value)

def current_subject_options():
    if scene_slider.value is None:
        return []
    return subjects_for(level_dd.value, scene_slider.value)

out = widgets.Output(layout=widgets.Layout(border="0px"))

# ---------- CALLBACKS ----------
def on_level_change(change):
    # rebuild scene slider with N positions for the selected level
    global scene_slider
    new_slider = build_scene_slider(level_dd.value)
    # swap the widget in place inside the UI
    controls.children = (level_dd, new_slider)
    # rebind handler
    new_slider.observe(on_scene_change, names="value")
    scene_slider.unobserve(on_scene_change, names="value")
    # update reference
    scene_slider = new_slider

    # update subject list
    subj_opts = current_subject_options()

    draw()

def on_scene_change(change):
    subj_opts = current_subject_options()
    draw()

def draw():
    with out:
        clear_output(wait=True)
        if (level_dd.value is None) or (scene_slider.value is None):
            print("No data available for the current selection.")
            return
        fig = make_dashboard(level_dd.value, scene_slider.value)
        # Use .show with minimal toolbar & no selection tools
        fig.show(config={"displaylogo": False, "modeBarButtonsToRemove": ["select", "lasso2d"]})

# Wire up observers
level_dd.observe(on_level_change, names="value")
scene_slider.observe(on_scene_change, names="value")

# ---------- DISPLAY FIRST, THEN DRAW (prevents initial overlap) ----------
controls = widgets.HBox([level_dd, scene_slider])
display(controls, out)
draw()  # clean initial render

Output(layout=Layout(border_bottom='0px', border_left='0px', border_right='0px', border_top='0px'))

In [110]:
def compute_delta_long(df, phase_to_ckpt):
    # copie + map phases -> chk
    df_tmp = df.copy()
    df_tmp["chk"] = df_tmp["Learning_Phase"].map(phase_to_ckpt)

    # mean clearance per Subject, chk
    df_chk = (
        df_tmp.groupby(["Subject", "chk"])["Cleared"]
        .mean()
        .reset_index(name="clearance")
    )

    # pivot : Subject x chk (wide)
    df_wide = df_chk.pivot(index="Subject", columns="chk", values="clearance")
    print(df_wide)
    df_wide = df_chk.pivot(index="Subject", columns="chk", values="clearance").fillna(0)#.ffill(axis=1).fillna(0)
    #print(df_wide)
    # shift vers la gauche : clearance_{k+1} aligned with column k
    df_shifted = df_wide.shift(-1, axis=1)
    print(df_shifted)

    # delta = shifted - original  -> delta at chk k = chk{k+1} - chk{k}
    df_delta = df_shifted - df_wide
    print(df_delta)

    # by construction, last column becomes NaN (no chk{k+1} for last)
    # ensure columns remain in same order

    # convert to long format and keep clearance too
    df_long_clear = df_delta.reset_index().melt(id_vars="Subject", var_name="chk", value_name="clearance").sort_values(["Subject", "chk"]).reset_index(drop=True)
    #print(df_long_clear)
    df_long_delta = df_delta.reset_index().melt(id_vars="Subject", var_name="chk", value_name="delta")
    #print(df_long_delta)

    # merge clearance + delta
    df_long = pd.merge(df_long_clear, df_long_delta, on=["Subject", "chk"])

    # optional : sort
    df_long = df_long.sort_values(["Subject", "chk"]).reset_index(drop=True)

    return df_long

level = 'w3l1'
scene = 4
df_test = get_subset(level, scene)
phase_to_human = {p: i % 4 for i, p in enumerate(human_phases)}
delta_human = compute_delta_long(df_test, phase_to_human)


chk      0.0  1.0  2.0  3.0
Subject                    
sub-02   NaN  NaN  1.0  NaN
sub-03   NaN  NaN  1.0  1.0
sub-05   1.0  1.0  1.0  NaN
sub-06   1.0  1.0  1.0  1.0
chk      0.0  1.0  2.0  3.0
Subject                    
sub-02   0.0  1.0  0.0  NaN
sub-03   0.0  1.0  1.0  NaN
sub-05   1.0  1.0  0.0  NaN
sub-06   1.0  1.0  1.0  NaN
chk      0.0  1.0  2.0  3.0
Subject                    
sub-02   0.0  1.0 -1.0  NaN
sub-03   0.0  1.0  0.0  NaN
sub-05   0.0  0.0 -1.0  NaN
sub-06   0.0  0.0  0.0  NaN
